In [66]:
import pickle

from collections import defaultdict
from scipy.stats import wilcoxon

METHOD_NAME_MAP = {
    "Louvain-Variant": "CTL",
    "Louvain-Variant Leiden-Addition": "CTL Leiden-Addition",
    "Louvain": "Louvain",
    "Temporal Louvain": "Temporal Louvain"
}

def return_def_list() -> defaultdict:
    """
    Function to create specific data structure that makes it easier to store multiple experiment runs
    """
    return defaultdict(list)

In [95]:
import copy

def get_stats(data_path):
    data = pickle.load(open(data_path, "rb"))
    results_orig = data["original"]
    data.pop("original")
    results_var = copy.deepcopy(data)

    stat_results = {}
    for cc_th, results in results_var.items():
        for method_name, results_method in results.items():
            for method_name_orig, results_method_orig in results_orig.items():
                stat_results[f"{method_name}_{method_name_orig}_{cc_th}"] = {
                    "time": wilcoxon(results_method["time"][0], results_method_orig["time"][0], alternative="less"),
                    "nmi": wilcoxon(results_method["nmi"][0], results_method_orig["nmi"][0], alternative="greater"),
                    "modularity": wilcoxon(results_method["modularity"][0], results_method_orig["modularity"][0], alternative="greater")
                }
            for method_name_var, results_method_var in results.items():
                if method_name_var == method_name:
                    continue
                stat_results[f"{method_name}_{method_name_var}_{cc_th}"] = {
                    "time": wilcoxon(results_method["time"][0], results_method_var["time"][0], alternative="less"),
                    "nmi": wilcoxon(results_method["nmi"][0], results_method_var["nmi"][0], alternative="greater"),
                    "modularity": wilcoxon(results_method["modularity"][0], results_method_var["modularity"][0], alternative="greater")
                }

    alpha = 0.05
    table_header = "\\hline\nMethod 1 & Method 2 & Metric & Hypothesis & Statistic & p-value \\\\ \\hline"
    row_template = "{method_1} & {method_2} & {metric_type} & {hypothesis} & {statistic} & {p} \\\\ "
    table_rows = [table_header]
    for pair_test, p in stat_results.items():
        method_x, method_y, cc_th = pair_test.split("_")
        method_x = METHOD_NAME_MAP[method_x]
        method_y = METHOD_NAME_MAP[method_y]
        
        if cc_th != "0.8":
            continue

        table_rows += [
            row_template.format(
                method_1=method_x,method_2=method_y,metric_type="Walltime",hypothesis="Less",statistic=p["time"].statistic,p=f"\\textbf{{{round(p["time"].pvalue,5)}}}" if round(p["time"].pvalue,5) < alpha else round(p["time"].pvalue,5)
            ),
            row_template.format(
                method_1=method_x,method_2=method_y,metric_type="NMI",hypothesis="Greater",statistic=p["nmi"].statistic,p=f"\\textbf{{{round(p["nmi"].pvalue,5)}}}" if round(p["nmi"].pvalue,5) < alpha else round(p["nmi"].pvalue,5)
            ),
            row_template.format(
                method_1=method_x,method_2=method_y,metric_type="Modularity",hypothesis="Greater",statistic=p["modularity"].statistic,p=f"\\textbf{{{round(p["modularity"].pvalue,5)}}}" if round(p["modularity"].pvalue,5) < alpha else round(p["modularity"].pvalue,5)
            ) + "\\midrule\\midrule"
        ]

    return table_rows

# Synthetic Network, 20% probability to switch

In [96]:
path = "results/results_100_5000_20_02_02.pickle"
table_rows = get_stats(path)

In [97]:
print("\n".join(table_rows))

\hline
Method 1 & Method 2 & Metric & Hypothesis & Statistic & p-value \\ \hline
CTL & Louvain & Walltime & Less & 0.0 & \textbf{0.0} \\ 
CTL & Louvain & NMI & Greater & 0.0 & 1.0 \\ 
CTL & Louvain & Modularity & Greater & 0.0 & 1.0 \\ \midrule\midrule
CTL & Temporal Louvain & Walltime & Less & 1402.0 & \textbf{6e-05} \\ 
CTL & Temporal Louvain & NMI & Greater & 1192.0 & 1.0 \\ 
CTL & Temporal Louvain & Modularity & Greater & 286.0 & 1.0 \\ \midrule\midrule
CTL & CTL Leiden-Addition & Walltime & Less & 1.0 & \textbf{0.0} \\ 
CTL & CTL Leiden-Addition & NMI & Greater & 0.0 & 1.0 \\ 
CTL & CTL Leiden-Addition & Modularity & Greater & 2.0 & 1.0 \\ \midrule\midrule
CTL Leiden-Addition & Louvain & Walltime & Less & 5049.0 & 1.0 \\ 
CTL Leiden-Addition & Louvain & NMI & Greater & 4944.0 & \textbf{0.0} \\ 
CTL Leiden-Addition & Louvain & Modularity & Greater & 0.0 & 1.0 \\ \midrule\midrule
CTL Leiden-Addition & Temporal Louvain & Walltime & Less & 5049.0 & 1.0 \\ 
CTL Leiden-Addition & Tempor

# Synthetic Network, 50% probability to switch

In [98]:
path = "results/results_100_5000_20_02_05.pickle"
table_rows = get_stats(path)

In [99]:
print("\n".join(table_rows))

\hline
Method 1 & Method 2 & Metric & Hypothesis & Statistic & p-value \\ \hline
CTL & Louvain & Walltime & Less & 0.0 & \textbf{0.0} \\ 
CTL & Louvain & NMI & Greater & 0.0 & 1.0 \\ 
CTL & Louvain & Modularity & Greater & 0.0 & 1.0 \\ \midrule\midrule
CTL & Temporal Louvain & Walltime & Less & 2406.0 & 0.34121 \\ 
CTL & Temporal Louvain & NMI & Greater & 519.0 & 1.0 \\ 
CTL & Temporal Louvain & Modularity & Greater & 773.0 & 1.0 \\ \midrule\midrule
CTL & CTL Leiden-Addition & Walltime & Less & 0.0 & \textbf{0.0} \\ 
CTL & CTL Leiden-Addition & NMI & Greater & 0.0 & 1.0 \\ 
CTL & CTL Leiden-Addition & Modularity & Greater & 1.0 & 1.0 \\ \midrule\midrule
CTL Leiden-Addition & Louvain & Walltime & Less & 5049.0 & 1.0 \\ 
CTL Leiden-Addition & Louvain & NMI & Greater & 4949.0 & \textbf{0.0} \\ 
CTL Leiden-Addition & Louvain & Modularity & Greater & 0.0 & 1.0 \\ \midrule\midrule
CTL Leiden-Addition & Temporal Louvain & Walltime & Less & 5050.0 & 1.0 \\ 
CTL Leiden-Addition & Temporal Louva